---

title: Watchdog 的使用
keywords: maixpy3, 看门狗, Watchdog
desc: maixpy3 doc: 看门狗, Watchdog

---

| 更新时间 | 负责人 | 内容 | 备注 |
| --- | --- | --- | --- |
| 2022年1月4日 | Rui | 初次编写文档 | --- |
| 2022年1月8日 | Rui | 修改文档的编写方式 | 使用 Jupyter notebook 进行编写文档 |
| 2022年1月18日 | Rui | 修改文档，增加效果图 | 目前测试的是 MaixII-Dock MaixPy3 0.4.0，其他平台理论可用，但未测试。 |

看门狗主要用于保护系统正常运行，作用原理为，看门狗启动后，程序中必须定时执行一个喂狗的操作，当系统受到干扰不能正常运行时，喂狗操作也不能定时执行，此时看门狗将产生内部复位，使系统重新开始工作。

## 代码

由于目前还在开发状态，以下代码为封装前代码，后续会有对应封装

In [ ]:
import array
import fcntl
import struct

IO_WRITE = 0x40000000
IO_READ = 0x80000000
IO_READ_WRITE = 0xC0000000
IO_SIZE_INT = 0x00040000
IO_SIZE_40 = 0x00280000
IO_TYPE_WATCHDOG = ord('W') << 8

WDR_INT = IO_READ | IO_SIZE_INT | IO_TYPE_WATCHDOG
WDR_40 = IO_READ | IO_SIZE_40 | IO_TYPE_WATCHDOG
WDWR_INT = IO_READ_WRITE | IO_SIZE_INT | IO_TYPE_WATCHDOG

WDIOC_GETSUPPORT = 0 | WDR_40
WDIOC_GETSTATUS = 1 | WDR_INT
WDIOC_GETBOOTSTATUS = 2 | WDR_INT
WDIOC_GETTEMP = 3 | WDR_INT
WDIOC_SETOPTIONS = 4 | WDWR_INT
WDIOC_KEEPALIVE = 5 | WDR_INT
WDIOC_SETTIMEOUT = 6 | WDWR_INT
WDIOC_GETTIMEOUT = 7 | WDR_INT
WDIOC_SETPRETIMEOUT = 8 | WDWR_INT
WDIOC_GETPRETIMEOUT = 9 | WDR_INT
WDIOC_GETTIMELEFT = 10 | WDR_INT

WDIOF_OVERHEAT = 0x0001
WDIOF_FANFAULT = 0x0002
WDIOF_EXTERN1 = 0x0004
WDIOF_EXTERN2 = 0x0008
WDIOF_POWERUNDER = 0x0010
WDIOF_CARDRESET = 0x0020
WDIOF_POWEROVER = 0x0040
WDIOF_SETTIMEOUT = 0x0080
WDIOF_MAGICCLOSE = 0x0100
WDIOF_PRETIMEOUT = 0x0200
WDIOF_ALARMONLY = 0x0400
WDIOF_KEEPALIVEPING = 0x8000

WDIOS_DISABLECARD = 0x0001
WDIOS_ENABLECARD = 0x0002
WDIOS_TEMPPANIC = 0x0004

WATCHDOG_DEVICE = '/dev/watchdog'
WATCHDOG_STOP = 'V'
WATCHDOG_START = 'S'


class Watchdog:
  def __init__(self, stop=True):
    super(Watchdog, self).__init__()
    self.fd = open(WATCHDOG_DEVICE, 'w')
    self.stop = stop
    if stop:
      self.Stop()

  def __del__(self) -> None: # for v831
    try:
      self.fd.close()
      import os
      os.system("echo V > " + WATCHDOG_DEVICE)
    except Exception as e:
      pass

  def Stop(self):
    self.fd.write(WATCHDOG_STOP)

  def Start(self):
    self.stop = False
    self.fd.write(WATCHDOG_START)

  def _IoctlInt(self, cmd, write_value=0):
    buf = array.array('I', [write_value])
    start_temporary = self.stop
    if start_temporary:
      self.Start()
    fcntl.ioctl(self.fd, cmd, buf, True)
    if start_temporary:
      self.Stop()
    return buf[0]

  def GetBootStatus(self):
    return self._IoctlInt(WDIOC_GETBOOTSTATUS)

  def GetTemp(self):
    return self._IoctlInt(WDIOC_GETTEMP)

  def SetOptions(self, options):
    return self._IoctlInt(WDIOC_SETOPTIONS, options)

  def KeepAlive(self):
    if self.stop:
      self.Start()
    return self._IoctlInt(WDIOC_KEEPALIVE)

  def SetTimeout(self, timeout):
    timeout = int(timeout)
    if timeout <= 0:
      raise ValueError('timeout <= 0')
    return self._IoctlInt(WDIOC_SETTIMEOUT, timeout)

  def GetTimeout(self):
    return self._IoctlInt(WDIOC_GETTIMEOUT)

  def SetPreTimeout(self, timeout):
    timeout = int(timeout)
    if timeout <= 0 or timeout > 16:
      raise ValueError('0 < timeout <= 16')
    return self._IoctlInt(WDIOC_SETPRETIMEOUT, timeout)

  def GetPreTimeout(self):
    return self._IoctlInt(WDIOC_GETPRETIMEOUT)

  def GetTimeLeft(self):
    return self._IoctlInt(WDIOC_GETTIMELEFT)

# 实例化看门狗 设置喂狗时间 1s
t = Watchdog()
t.SetTimeout(1)
import time

# 进行喂狗操作 0.5s 喂一次
t.KeepAlive()
time.sleep(0.5)
t.KeepAlive()
time.sleep(0.5)
t.__del__()
del t

# 实例化看门狗 设置喂狗时间 2s
t = Watchdog()
t.SetTimeout(2)
time.sleep(2)
del t

当喂狗失败之后，系统会自动昂重启运行效果如下：

![](./../asserts/Watchdog.gif)

## 了解更多
查看[大佬鼠博客](https://www.cnblogs.com/juwan/p/14870346.html)中的关于看门狗的介绍